In [2]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 19.7 MB 3.0 MB/s eta 0:00:01     |█████████████████████████▎      | 15.6 MB 3.4 MB/s eta 0:00:02
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
     |████████████████████████████████| 58 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 471 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 4.0 MB/s 

In [3]:
from huggingface_hub import hf_hub_download

/home/ellewong/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path = hf_hub_download(repo_id="TheBloke/WizardLM-Uncensored-SuperCOT-StoryTelling-30B-GGML", filename="WizardLM-Uncensored-SuperCOT-Storytelling.ggmlv3.q3_K_S.bin")

In [5]:
path

'/home/ellewong/.cache/huggingface/hub/models--TheBloke--WizardLM-Uncensored-SuperCOT-StoryTelling-30B-GGML/snapshots/dfc28ffcd7863fca495ea5b29a6ffd487937ccd0/WizardLM-Uncensored-SuperCOT-Storytelling.ggmlv3.q3_K_S.bin'

In [6]:
from llama_cpp import Llama

In [7]:
llm = Llama(model_path=path, n_ctx=2048)

llama.cpp: loading model from /home/ellewong/.cache/huggingface/hub/models--TheBloke--WizardLM-Uncensored-SuperCOT-StoryTelling-30B-GGML/snapshots/dfc28ffcd7863fca495ea5b29a6ffd487937ccd0/WizardLM-Uncensored-SuperCOT-Storytelling.ggmlv3.q3_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32001
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 6656
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 52
llama_model_load_internal: n_layer    = 60
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 11 (mostly Q3_K - Small)
llama_model_load_internal: n_ff       = 17920
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 30B
llama_model_load_internal: ggml ctx size =    0.13 MB
llama_model_load_internal: mem required  = 15636.65 MB (+ 3124.00 MB per state)
......................................................

In [74]:
prompt = """## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Get webpage summary
  - *Description*: Delegate to a separate AI to read the full content of a webpage, and have it provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [get_webpage_summary(<url>, <question>)]
  - *Example*: [get_webpage_summary("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge, then decide on a course of action. Describe verbally your decision but do not output tool use yet.>
*Act 1*: <based on your decision in the thought above, output should be a tool use strictly adhering to the formats in "Usage". make no other outputs here and no markdown codeblock necessary.>
*Observation 1*: <result of tool use will be shown here>

This pattern can be repeated (with the number increasing such as *Thought 2*, *Thought 3*, etc. Ditto for Act and Observation) until you arrive at a final answer.

You should still output the observation line when your action is to finalize your answer.

You are not allowed to ask the user in the middle of your thought-stream - try to make decision by yourself until you are ready to give a full and final answer.

In particular, you are advised to follow the following strategy:
- First judge if you can immediately answer the user yourself. This might be the case if user is doing a greeting, social chit-chat, or anything that does not involves a need to obtain new or up-to-date information.
- If not, then you should perform an appropiate web search.
- Then, in the next thought, you should use the search ranking returned and the text snippet provided of each search result, to choose which document to read in more detail by the other AI, using the "Get webpage summary" tool.
- You can adjust the question given to this tool to selectively focus on different aspect of the document.
- You can repeat this up to three times (due to time constraint)
- Then, provide a final answer by consolidating all the information you have obtained.

## Input
User: Give me a summary of news on debt ceiling in the US.
## Output
Assistent: Let's think step by step.
"""

In [12]:
myOut = llm(prompt, max_tokens=200, stop=["*Observation"], echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 317036.69 ms
llama_print_timings:      sample time =    64.53 ms /    47 runs   (    1.37 ms per token,   728.33 tokens per second)
llama_print_timings: prompt eval time = 189390.70 ms /   459 tokens (  412.62 ms per token,     2.42 tokens per second)
llama_print_timings:        eval time = 33105.80 ms /    46 runs   (  719.69 ms per token,     1.39 tokens per second)
llama_print_timings:       total time = 222806.52 ms


In [14]:
response = myOut["choices"][0]["text"]

In [15]:
print(response)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Read and summarize webpage
  - *Description*: Read the content of a webpage. A separate AI will provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [read_summarize_webpage(<url>, <question>)]
  - *Example*: [read_summarize_webpage("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge,

In [57]:
thought1 = """*Thought 1*: The user wants a summary of news on debt ceiling in the US.
*Act 1*: [web_search("debt ceiling news US")]"""

In [17]:
response.split("\n")[-2]

'*Act 1*: [web_search("debt ceiling news US")]'

In [18]:
import re

In [97]:
action_regex = "(web_search|get_webpage_summary|final_ans)\(\"(.*)\"\)"
regex_result = re.search(action_regex, response.split("\n")[-2])

In [23]:
regex_result[2]

'debt ceiling news US'

In [24]:
from duckduckgo_search import DDGS

In [26]:
!pip3 install httpx[http2]

     |████████████████████████████████| 57 kB 1.8 MB/s eta 0:00:01


In [27]:
from itertools import islice

#region='wt-wt', safesearch='Off', timelimit='y'
#backend is api, html, or lite
with DDGS() as ddgs:
    ddgs_gen = ddgs.text("notes from a dead house", backend="lite")
    for r in islice(ddgs_gen, 10):
        print(r)

{'title': 'The House of the Dead (novel) - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/The_House_of_the_Dead_(novel)', 'body': 'The House of the Dead ( Russian: Записки из Мёртвого дома, Zapiski iz Myortvovo doma) is a semi-autobiographical novel published in 1860-2 [1] in the journal Vremya [2] by Russian author Fyodor Dostoevsky.'}
{'title': 'Notes from a Dead House - Penguin Random House', 'href': 'https://www.penguinrandomhouse.com/books/218281/notes-from-a-dead-house-by-fyodor-dostoevsky-translated-by-richard-pevear-and-larissa-volokhonsky/', 'body': 'Notes from a Dead House (sometimes translated as The House of the Dead) depicts brutal punishments, feuds, betrayals, and the psychological effects of confinement, but it also reveals the moments of comedy and acts of kindness that Dostoevsky witnessed among his fellow prisoners. To get past government censors, Dostoevsky made his narrator a ...'}
{'title': 'Notes from a Dead House (Vintage Classics) - amazon.com', 'href': 'ht

In [29]:
search_result = None
with DDGS() as ddgs:
    ddgs_gen = ddgs.text(regex_result[2], backend="lite")
    search_result = [r for r in islice(ddgs_gen, 10)]

In [33]:
search_result[0]

{'title': "Debt ceiling: What's next for the US debt limit - BBC News",
 'href': 'https://www.bbc.com/news/world-us-canada-58820071',
 'body': 'US lawmakers have temporarily put off a dangerous game of brinkmanship over lifting the debt ceiling - a limit on how much the US government can borrow. Treasury Secretary Janet Yellen had warned ...'}

In [35]:
"Hello {name}, {greeting}".format(name="Tom", greeting="welcome!")

'Hello Tom, welcome!'

In [37]:
print("{n}. [{title}]({url})\n{body}".format(n=1, title=search_result[0]["title"], url=search_result[0]["href"], body=search_result[0]["body"]))

1. [Debt ceiling: What's next for the US debt limit - BBC News](https://www.bbc.com/news/world-us-canada-58820071)
US lawmakers have temporarily put off a dangerous game of brinkmanship over lifting the debt ceiling - a limit on how much the US government can borrow. Treasury Secretary Janet Yellen had warned ...


In [41]:
print("\n".join(["{n}. [{title}]({url})\n{body}".format(n=i+1, title=search_result[i]["title"], url=search_result[i]["href"], body=search_result[i]["body"]) for i in range(0, 10)]))

1. [Debt ceiling: What's next for the US debt limit - BBC News](https://www.bbc.com/news/world-us-canada-58820071)
US lawmakers have temporarily put off a dangerous game of brinkmanship over lifting the debt ceiling - a limit on how much the US government can borrow. Treasury Secretary Janet Yellen had warned ...
2. [Mike Quigley: Debt ceiling deal was better than US defaulting](https://www.chicagotribune.com/opinion/commentary/ct-opinion-debt-ceiling-bill-default-bipartisan-20230620-4raix4a2bvckzdhxlcdcukncmu-story.html)
Mike Quigley: The debt ceiling deal was not perfect, but it was far better than a default. Senate Majority Leader Chuck Schumer, D-N.Y., after the Senate voted 63-36 late on June 1, 2023, to ...
3. [The US debt-ceiling 'deal' was a giant exercise in bipartisan class ...](https://www.theguardian.com/commentisfree/2023/jun/14/us-debt-ceiling-bipartisan-class-warfare)
60 T he headlines around the debt-ceiling legislation focused on the ability of the US to meet its finan

In [86]:
search_results_formatted = "\n".join(["{n}. [{title}]({url})\n{body}".format(n=i+1, title=search_result[i]["title"], url=search_result[i]["href"], body=search_result[i]["body"]) for i in range(0, 5)])

In [46]:
new_query = response + "*Observation 1*: Search Result:\n" + search_results_formatted + "\n\n"
print(new_query)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Read and summarize webpage
  - *Description*: Read the content of a webpage. A separate AI will provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [read_summarize_webpage(<url>, <question>)]
  - *Example*: [read_summarize_webpage("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge,

In [50]:
myOut = llm(new_query + "*Thought 2*: ", max_tokens=200, stop=["*Observation"], echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 317036.69 ms
llama_print_timings:      sample time =    49.32 ms /    54 runs   (    0.91 ms per token,  1094.93 tokens per second)
llama_print_timings: prompt eval time =  3085.19 ms /     7 tokens (  440.74 ms per token,     2.27 tokens per second)
llama_print_timings:        eval time = 34676.39 ms /    53 runs   (  654.27 ms per token,     1.53 tokens per second)
llama_print_timings:       total time = 38242.33 ms


In [51]:
response2 = myOut["choices"][0]["text"]

In [52]:
print(response2)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Read and summarize webpage
  - *Description*: Read the content of a webpage. A separate AI will provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [read_summarize_webpage(<url>, <question>)]
  - *Example*: [read_summarize_webpage("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge,

In [53]:
myOut = llm(response2 + "\n*Act 2*: ", max_tokens=200, stop=["*Observation"], echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 317036.69 ms
llama_print_timings:      sample time =    30.86 ms /    39 runs   (    0.79 ms per token,  1263.85 tokens per second)
llama_print_timings: prompt eval time =  2886.76 ms /     8 tokens (  360.84 ms per token,     2.77 tokens per second)
llama_print_timings:        eval time = 23498.98 ms /    38 runs   (  618.39 ms per token,     1.62 tokens per second)
llama_print_timings:       total time = 26833.90 ms


In [54]:
response3 = myOut["choices"][0]["text"]

In [55]:
print(response3)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Read and summarize webpage
  - *Description*: Read the content of a webpage. A separate AI will provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [read_summarize_webpage(<url>, <question>)]
  - *Example*: [read_summarize_webpage("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge,

In [87]:
test_query = prompt + thought1 + "\n*Observation 1*: Search Result:\n" + search_results_formatted + "\n*System Hint*: Which of the search result above appears to warrant further reading to obtain information to answer user's question?\n" + "*Thought 2*: "

In [88]:
print(test_query)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Get webpage summary
  - *Description*: Delegate to a separate AI to read the full content of a webpage, and have it provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [get_webpage_summary(<url>, <question>)]
  - *Example*: [get_webpage_summary("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and y

In [90]:
myOut = llm(test_query, max_tokens=200, stop=["*Observation"], echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 317036.69 ms
llama_print_timings:      sample time =   171.52 ms /   143 runs   (    1.20 ms per token,   833.75 tokens per second)
llama_print_timings: prompt eval time = 19069.64 ms /    37 tokens (  515.40 ms per token,     1.94 tokens per second)
llama_print_timings:        eval time = 112909.90 ms /   142 runs   (  795.14 ms per token,     1.26 tokens per second)
llama_print_timings:       total time = 133563.55 ms


In [91]:
print(myOut["choices"][0]["text"])

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Get webpage summary
  - *Description*: Delegate to a separate AI to read the full content of a webpage, and have it provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [get_webpage_summary(<url>, <question>)]
  - *Example*: [get_webpage_summary("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and y

In [68]:
response_test1 = myOut["choices"][0]["text"]

In [85]:
myOut

{'id': 'cmpl-72c244a8-d2e5-485c-bbfc-1e2023c0ade7',
 'object': 'text_completion',
 'created': 1687444326,
 'model': '/home/ellewong/.cache/huggingface/hub/models--TheBloke--WizardLM-Uncensored-SuperCOT-StoryTelling-30B-GGML/snapshots/dfc28ffcd7863fca495ea5b29a6ffd487937ccd0/WizardLM-Uncensored-SuperCOT-Storytelling.ggmlv3.q3_K_S.bin',
 'choices': [{'text': '## Instruction\nYou are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:\n\n- Web Search\n  - *Description*: Use search engine online to find webpage.\n  - *Usage*: [web_search(<query>)]\n  - *Example*: [web_search("how to make a cake")]\n- Get webpage summary\n  - *Description*: Delegate to a separate AI to read the full content of a webpage, and have it provide a summary of the webpage\'s content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".\n  - *Usage*: [g

In [69]:
print(response_test1)

## Instruction
You are a personal assistent that answer user query in the context of a multi-round conversation. You have access to the following tools to help you:

- Web Search
  - *Description*: Use search engine online to find webpage.
  - *Usage*: [web_search(<query>)]
  - *Example*: [web_search("how to make a cake")]
- Read and summarize webpage
  - *Description*: Read the content of a webpage. A separate AI will provide a summary of the webpage's content that are pertinent to a question you pose to it. If question is not provided it will default to "Provide a summary of the text".
  - *Usage*: [read_summarize_webpage(<url>, <question>)]
  - *Example*: [read_summarize_webpage("https://apple.com/", "What are the latest products on offer?")]
- Finalize your answer
  - *Description*: Indicate that you are ready to answer the user.
  - *Usage*: [final_ans()]

You should think using the following pattern:

*Thought 1*: <your internal thoughts based on current state and your knowledge,

In [63]:
test_thought2 = """*Thought 2*: 
Act 2: [summarize_news("debt ceiling news US")](https://www.bbc.com/news/world-us-canada-58820071)
The US Congress has temporarily raised the debt ceiling, avoiding a dangerous game of brinkmanship over how much the government can borrow. The deal, which was passed by both houses of Congress and signed by President Biden, will allow the Treasury Department to meet its financial obligations on time and in full through 2024. However, some Democrats are wary of another battle in just a few years, while Republicans want deeper spending cuts. The deal also includes provisions for disaster relief funding and an increase in the gas tax to pay for infrastructure projects."""

In [73]:
test_thought3 = """*Thought 2*: 
Based on the search results, it seems that the US has temporarily avoided a dangerous game of brinkmanship over lifting the debt ceiling through a bipartisan deal that allows the government to borrow up to $3."""

In [92]:
proceed_result = myOut["choices"][0]["text"]

In [94]:
proceed_result.split("\n")[-2]

'*Act 2*: [get_webpage_summary("https://www.bbc.com/news/world-us-canada-58820071")]'

In [98]:
regex_result2 = re.search(action_regex, proceed_result.split("\n")[-2])

In [101]:
regex_result2[2]

'https://www.bbc.com/news/world-us-canada-58820071'

In [102]:
import requests
import html2text

text_maker = html2text.HTML2Text()
text_maker.ignore_links = True
text_maker.ignore_images = True

def grab_webpage(url, text_maker):
    req = requests.get(url)
    return text_maker.handle(req.text)

In [104]:
doc = grab_webpage(regex_result2[2], text_maker)

In [106]:
print(doc)

BBC Homepage

  * Skip to content
  * Accessibility Help

  * Your account

  * Home
  * News
  * Sport
  * Reel
  * Worklife
  * Travel
  * Future
  * More menu

More menu

Search BBC

  * Home
  * News
  * Sport
  * Reel
  * Worklife
  * Travel
  * Future
  * Culture
  * Music
  * TV
  * Weather
  * Sounds

Close menu

BBC News

Menu

  * Home
  * War in Ukraine
  * Coronavirus
  * Climate
  * Video
  * World
  * Asia
  * UK
  * Business
  * Tech

More

  * Science
  * Entertainment & Arts
  * Health
  * World News TV
  * In Pictures
  * Reality Check
  * Newsbeat

  * World
  * Africa
  * Australia
  * Europe
  * Latin America
  * Middle East
  * US & Canada

# Debt ceiling: What's next for the US debt limit

  * Published

15 October 2021

Share

close panel

Share page

Copy link

About sharing

Image source, Getty Images

Image caption,

US lawmakers are running out the clock in a standoff over how much money the
government can borrow

 **US lawmakers have temporarily put off a d

In [112]:
tDoc = llm.tokenize(bytes(doc, "utf-8"))

llama_tokenize: too many tokens


In [124]:
len(tDoc)

2975

In [149]:
chunk_size = 800
chunk_overlap = 30

chunks = []
b = 0
reachedEnd = False
while not reachedEnd:
    reachedEnd = not (b + chunk_size < len(tDoc))
    cur_chunk = llm.detokenize(tDoc[b:b + chunk_size]).decode("utf-8")
    chunks.append(cur_chunk)
    b = b + chunk_size - chunk_overlap

In [131]:
ttt = [0, 1, 2, 3, 4]
ttt[2:6]

[2, 3, 4]

In [150]:
len(chunks)

4

In [151]:
refine_prompt = """### Instruction
Consider this question or request:

{question}

Input section below contains:
- Part of the text of a webpage (also called a snippet)
- (Optionally) a partial answer to the question or request above based on reading previous parts of the webpage.

Create or refine a partial answer towards answering the question or request based on any new info contained in the text of the webpage provided. Write your updated partial answer in the output section. If you did not find any useful info in the snippet, please simply copy the existing partial answer in input section to the output section.

### Input
Part of the text of webpage (snippet):
{snippet}
----
Partial answer so far (can be empty):
{partialans}
### Output
*Updated partial answer*: """

In [ ]:
main_question = "Provide a summary of the article towards answering the question: Give me a summary of news on debt ceiling in the US."

refine_prompt_formated = refine_prompt.format(question=main_question,\
                    snippet=chunks[1],\
                    partialans=cur_ans)

print(refine_prompt_formated)

In [154]:
myOut = llm(refine_prompt_formated, max_tokens=200, echo=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 317036.69 ms
llama_print_timings:      sample time =   203.38 ms /   145 runs   (    1.40 ms per token,   712.96 tokens per second)
llama_print_timings: prompt eval time = 522092.71 ms /   996 tokens (  524.19 ms per token,     1.91 tokens per second)
llama_print_timings:        eval time = 111174.30 ms /   144 runs   (  772.04 ms per token,     1.30 tokens per second)
llama_print_timings:       total time = 634348.02 ms


In [157]:
cur_ans = myOut["choices"][0]["text"].split("\n")[-1]
print(cur_ans)

The US debt ceiling is a limit on how much the government can borrow to make up for the shortfall between its spending and tax revenues. On October 15, 2021, Congress temporarily extended the debt ceiling through early December, avoiding a potential default on the national debt. The extension was passed with bipartisan support and signed into law by President Joe Biden. While a default has never happened in US history, a potential default would have catastrophic implications for the US and global economy. The debt ceiling is a controversial topic in Washington politics, often used as a bargaining chip in negotiations between parties.
